In [61]:
import json
import pandas as pd
import codecs
import math
from pandas.io.json import json_normalize as jsy
#!pip install gensim
from gensim.models import Word2Vec
from nltk.corpus import brown # , #movie_review
import numpy as np
from textblob import TextBlob as tb

In [28]:
dev_set = pd.read_json('C:/Users/gokul/Documents/Projects/selqa-evaluater/SelQA-ass-dev.json', lines=True)
train_set = pd.read_json('C:/Users/gokul/Documents/Projects/selqa-evaluater/SelQA-ass-train.json', lines = True)
test_set = pd.read_json('C:/Users/gokul/Documents/Projects/selqa-evaluater/SelQA-ass-test.json', lines = True)

In [29]:
train_set.iloc[999]


article                                                    Physics
candidates                                                     [3]
is_paraphrase                                                 True
question         What's an example of something that was a prob...
section                                             Modern physics
sentences        [Modern physics began in the early 20th centur...
type                                                       SCIENCE
Name: 999, dtype: object

### following is helper cell no need to run for exposure purpose

In [ ]:
test_set[test_set['article']=='Table tennis']
x = test_set.loc[1422]['sentences']
train_set[(train_set['article']=='Table tennis')&(train_set['section']=='Ball')]

temp = (train_set.loc[0]['sentences'])
j=0
for i in range(len((train_set.loc[0]['sentences']))):
    print(j)
    print(temp[i])
    j+=1
    
## Unwanted can be deleted later
b = Word2Vec(brown.sents())
b.most_similar('money', topn=5)

type(brown.sents())
x = list(brown.sents())
len(x)
len(x[10])
x[10]
b.build_vocab_from_freq(word_freq=3)

### Create new dataframe from origine - mapping question to answer yet to complete for more than 1 candidate keys

In [30]:
train = pd.DataFrame(train_set['question'])
train['answers'] = "NA"
train.shape

temp1 = len(train_set)
for i in range(temp1):
    #print(type(train_set.loc[i]['question']))
    temp = len(train_set.loc[i]['candidates'])
    if temp==1:
        anskey = train_set.loc[i]['candidates'][0]
        #print(type(train_set.loc[i]['sentences'][anskey]))
        train.loc[i]['answers'] = train_set.loc[i]['sentences'][anskey]
    else:
        pass

In [31]:
writetrain = train[train['answers']!="NA"]
X_train = np.array(writetrain['question'])
Y_train = np.array(writetrain['answers'])

maxLenxt = len(max(X_train, key=len).split())
maxLenyt = len(max(Y_train, key=len).split())
minLenxt = len(min(X_train, key=len).split())
minLenyt = len(min(Y_train, key=len).split())
print("Max lenght of X training:", maxLenxt)
print("Max lenght of Y training:", maxLenyt)
print("Min lenght of X training:", minLenxt)
print("Min lenght of Y training:", minLenyt)

Max lenght of X training: 31
Max lenght of Y training: 83
Min lenght of X training: 3
Min lenght of Y training: 1


In [32]:
def read_glove_vec(glove_file):
    with open(glove_file,encoding='utf8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [33]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vec('C:/Users/gokul/Documents/Deep_learning/New folder/C5W2A2 - Emojify - v2/data/glove.6B.50d.txt')

word = "expired"
index = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])

the index of expired in the vocabulary is 142847
the 289846th word in the vocabulary is potatos


### Removing Question mark in all sentences

In [35]:
count = 0
countn = 0
for i in range(len(X_train)):
    temp = X_train[i][-1]
    if temp != "?":
        pass
    else:
        X_train[i] = X_train[i][:-1]
        temp = X_train[i][-1]
        if temp == "?":
            X_train[i] = X_train[i][:-1]
     
### validating if all "?" are removed
count = 0
countn = 0
for i in range(len(X_train)):
    temp = X_train[i][-1]
    if temp != "?":
        #print(X_train[i])
        count+=1
    else:
        countn+=1
        print(i)
        print(X_train[i])
        #X_train[i] = X_train[i][:-1]
print(count)
print(countn)

5079
0


In [36]:
def sentences_to_indices(X, word_to_vec_map, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    
    m = X.shape[0]                                   # number of training examples
    
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m,max_len))
    X_embedings = np.zeros((m,max_len,50))
    te = list(word_to_vec_map.keys())
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        temp1 = tb(X[i])
        temp2 = tb(str(temp1.lower()))
        sentence_words =temp2.words
#         print(sentence_words)
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
#         print(i,j)
#         print(word_to_vec_map['during'])
        for w in sentence_words:
#             print(w)
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            if str(w) in te:
                X_embedings[i, j] = word_to_vec_map[str(w)]
                X_indices[i,j] = word_to_index[str(w)]
                
            j = j + 1
        
    return X_indices, X_embedings

In [37]:
X, X_embedings = sentences_to_indices(X_train, word_to_vec_map, 35)
Y,Y_embedings = sentences_to_indices(Y_train, word_to_vec_map, 100)

In [43]:
print(X.shape)
print(Y.shape)
print(X_embedings.shape)
print(Y_embedings.shape)

(5079, 35)
(5079, 100)
(5079, 35, 50)
(5079, 100, 50)


In [44]:
np.save('C:/Users/gokul/Documents/Projects/selqa-evaluater/X.npy', X)
np.save('C:/Users/gokul/Documents/Projects/selqa-evaluater/Y.npy',Y)
np.save('C:/Users/gokul/Documents/Projects/selqa-evaluater/X_embedings.npy',X_embedings)
np.save('C:/Users/gokul/Documents/Projects/selqa-evaluater/Y_embedings.npy',Y_embedings)
#new = np.load('./X.npy') to load back 

In [45]:
X = np.load('C:/Users/gokul/Documents/Projects/selqa-evaluater/X.npy')
Y = np.load('C:/Users/gokul/Documents/Projects/selqa-evaluater/Y.npy')
X_embedings = np.load('C:/Users/gokul/Documents/Projects/selqa-evaluater/X_embedings.npy')
Y_embedings = np.load('C:/Users/gokul/Documents/Projects/selqa-evaluater/Y_embedings.npy')


In [11]:
print(X.shape)
print(Y.shape)

(5079, 35, 50)
(5079, 100, 50)


In [102]:
x_seq_length = len(X[1])
y_seq_length = len(Y[1])- 1
print(y_seq_length)


99


In [100]:
def random_mini_batches(X, Y, X_embedings, Y_embedings, mini_batch_size = 128, seed = 0):
    
    X = X.T
    Y = Y.T
    #print(Y.shape)
    m = X.shape[1]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation] ##.reshape((Y.shape[0],m))
    shuffled_X_embedings = X_embedings[permutation,:]
    shuffled_Y_embedings = Y_embedings[permutation,:]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_X_emb = shuffled_X_embedings[ k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch_Y_emb = shuffled_Y_embedings[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X.T, mini_batch_Y.T, mini_batch_X_emb,mini_batch_Y_emb )
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_X_emb = shuffled_X_embedings[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch_Y_emb = shuffled_Y_embedings[num_complete_minibatches * mini_batch_size : m, :]
        mini_batch = (mini_batch_X.T, mini_batch_Y.T, mini_batch_X_emb, mini_batch_Y_emb)
        mini_batches.append(mini_batch)
    
    return mini_batches


In [103]:
tem = random_mini_batches(X,Y, X_embedings, Y_embedings)
print(len(tem))
tem1 = tem[1]
tem2,tem3,tem4,tem5 = tem1
print(tem2.shape)
print(tem3.shape)
print(tem4.shape)
print(tem5.shape)

40
(128, 35)
(128, 100)
(128, 35, 50)
(128, 100, 50)


In [118]:
epochs = 2
# batch_size = 128
minibatch_size = 128
nodes = 128
embed_size = 10
import tensorflow as tf

tf.reset_default_graph()
sess = tf.InteractiveSession()

inputs = tf.placeholder(tf.int32, (None, x_seq_length), 'inputs')
outputs = tf.placeholder(tf.int32, (None, None), 'output')
targets = tf.placeholder(tf.int32, (None, None), 'targets')

ques_input_embedding = tf.placeholder(tf.float32, (None, 35,50))
ques_output_embedding = tf.placeholder(tf.float32,(None, None, 50))

with tf.variable_scope("encoding") as encoding_scope:
    lstm_enc = tf.contrib.rnn.BasicLSTMCell(nodes)
    _, last_state = tf.nn.dynamic_rnn(lstm_enc, inputs=ques_input_embedding, dtype=tf.float32)

with tf.variable_scope("decoding") as decoding_scope:
    lstm_dec = tf.contrib.rnn.BasicLSTMCell(nodes)
    dec_outputs, _ = tf.nn.dynamic_rnn(lstm_dec, inputs=ques_output_embedding, initial_state=last_state)
    
logits = tf.contrib.layers.fully_connected(dec_outputs, num_outputs=100, activation_fn=None) 
with tf.name_scope("optimization"):
    print(logits.get_shape().as_list())
    print(targets.get_shape().as_list())
    loss = tf.contrib.seq2seq.sequence_loss(logits, targets, tf.ones([minibatch_size, 100]))
    optimizer = tf.train.RMSPropOptimizer(1e-3).minimize(loss)


[None, None, 100]
[None, None]


In [116]:
print(dec_outputs.get_shape().as_list())
print(last_state[0].get_shape().as_list())
print(ques_input_embedding.get_shape().as_list())
print(logits.get_shape().as_list())
print(targets.get_shape().as_list())

[None, None, 128]
[None, 128]
[None, 35, 50]
[None, None, 100]
[None, None]


In [120]:
import random
import json
import os
import time

sess.run(tf.global_variables_initializer())
epochs = 10
for epoch_i in range(epochs):
    start_time = time.time()
    m = 5079
    num_minibatches = int(m / minibatch_size)
    minibatches = random_mini_batches(X, Y, X_embedings, Y_embedings)
    
    
    for minibatch in minibatches:
        (minibatch_X, minibatch_Y, minibatch_X_emb, minibatch_Y_emb) = minibatch
        #print(minibatch_Y[:, 1:].shape)
        _, batch_loss, batch_logits = sess.run([optimizer, loss, logits],
            feed_dict = {inputs: minibatch_X,
             outputs: minibatch_Y[:, :-1],
             targets: minibatch_Y[:, 1:],
             ques_input_embedding: minibatch_X_emb,
             ques_output_embedding: minibatch_Y_emb})
    
#     for batch_i, (source_batch, target_batch) in enumerate(batch_data(X, Y, batch_size)):
#         _, batch_loss, batch_logits = sess.run([optimizer, loss, logits],
#             feed_dict = {inputs: source_batch,
#              outputs: target_batch[:, :-1],
#              targets: target_batch[:, 1:]})
#     accuracy = np.mean(batch_logits.argmax(axis=-1) == target_batch[:,1:])
    print('Epoch {:3} Loss: {:>6.3f} Accuracy: {:>6.4f} Epoch duration: {:>6.3f}s'.format(epoch_i, batch_loss, 
                                                                      100, time.time() - start_time))

InvalidArgumentError: logits and labels must have the same first dimension, got logits shape [12800,100] and labels shape [12672]
	 [[Node: optimization/sequence_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](optimization/sequence_loss/Reshape, optimization/sequence_loss/Reshape_1)]]

Caused by op 'optimization/sequence_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits', defined at:
  File "C:\Users\gokul\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\gokul\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\gokul\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\gokul\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in laun